In [2]:
import pandas as pd

In [9]:
dat = pd.read_csv("full_contracts_dataset_CLEANED.csv")

In [10]:
dat.head()

,Unnamed: 0,Doge Agency,Doge Upload Date,Contract URL,Doge Value,Doge Desc,Valid URL,agencyID,PIID,modNumber,...,emailAddress,individualOrderLimit,fixedFeeValue,feeRangeLowerValue,feeRangeUpperValue,orderingProcedure,displayIDVType,typeOfIDC,multipleOrSingleAwardIDC,whoCanUse
0,0,COMMODITY FUTURES TRADING COMMISSION,2/11/2025,https://www.fpds.gov/ezsearch/jsp/viewLinkCont...,0.0,RENEW WEST PRINT SUBSCRIPTIONS FOR VARIOUS LEG...,True,9507,9523ZY19F0048,P00001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,COMMODITY FUTURES TRADING COMMISSION,2/13/2025,https://www.fpds.gov/ezsearch/jsp/viewLinkCont...,60373.0,ANNUAL SUBSCRIPTION TO POLITICO PRO,True,9507,9523ZY21P0041,P00003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,CONSUMER FINANCIAL PROTECTION BUREAU,2/11/2025,https://www.fpds.gov/ezsearch/jsp/viewLinkCont...,148350.0,BLOOMBERG TERMINALS,True,955F,9531CB24C0024,P00001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,CONSUMER FINANCIAL PROTECTION BUREAU,2/11/2025,https://www.fpds.gov/ezsearch/jsp/viewLinkCont...,584172.0,BLOOMBERG GOVERNMENT SUBSCRIPTIONS,True,955F,9531CB23F0100,P00002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,CONSUMER FINANCIAL PROTECTION BUREAU,2/11/2025,https://www.fpds.gov/ezsearch/jsp/viewLinkCont...,522660.0,POLITICO PRO,True,955F,9531CB25P0012,P00001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Helper function to cleanly print dollar figures.
def format_as_money(amount: float) -> str:
    """Formats a float as a string representing money, 
    including commas and two decimal places."""
    return "${:,.2f}".format(amount)

# Calculate DOGE claimed savings

Looking only at the values posted on the "Wall of Receipts" , how much is being claimed in savings?

In [12]:
NUM_CONTRACTS_TOTAL = len(dat)
DOGE_CLAIMED_SAVINGS = sum(dat['Doge Value'])
DOGE_CLAIMED_SAVINGS = format_as_money(DOGE_CLAIMED_SAVINGS)
print(DOGE_CLAIMED_SAVINGS)

$7,186,461,223.00


# How much was saved by calculating the contract value minus money already spent on the contract

The government cannot retrieve these costs. This is calculated by subtracting the base and all exercised options total from the total contract value.

Negative exercised options - typically money going away from a contract
If negative and greater than TCV, ignore it.



In [13]:
def calculate_savings(row):
    tcv = row['totalUltimateContractValue']
    spent = row['totalBaseAndExercisedOptionsValue']
    return tcv - spent

dat['maximumSavingsRealized'] = dat.apply(lambda row: calculate_savings(row), axis=1)


MAX_SAVINGS_ACTUALS = (sum(dat['maximumSavingsRealized']))
MAX_SAVINGS_ACTUALS = format_as_money(MAX_SAVINGS_ACTUALS)
MAX_SAVINGS_ACTUALS

'$3,400,312,355.37'

# Filter out rows that were modified before the inaguration date.

DOGE cannot claim savings from contracts that were last actioned prior to Donald Trump's election, as DOGE did not exist.

In [8]:
dat['displayLastModifiedDate'] = pd.to_datetime(dat['displayLastModifiedDate'], format="%m/%d/%Y %H:%M:%S")

inaguration = pd.to_datetime("1/20/2025")
not_cancelled = dat[dat['displayLastModifiedDate'] < inaguration]
dat = dat[dat['displayLastModifiedDate'] >= inaguration]

NUM_CONTRACTS_AFTER_INAGURATION = len(dat)
MAX_SAVINGS_ACTUALS_AFTER_INAGURATION = sum(dat['maximumSavingsRealized'])
MAX_SAVINGS_ACTUALS_AFTER_INAGURATION = format_as_money(MAX_SAVINGS_ACTUALS_AFTER_INAGURATION)
MAX_SAVINGS_ACTUALS_AFTER_INAGURATION

'$1,491,315,208.41'

In [16]:
not_cancelled[['PIID', 'Contract URL', 'displayLastModifiedDate']].to_csv("modified_before_inaguration.csv")

In [69]:
impossible_savings_statuses = [
    "Exercise an Option",
    "Supplemental Agreement for work within scope",
    "Additional Work (New Agreement, Justification Required"
]

def natural_closeout(closeOutDate, awardCompletionDate):
    return (closeOutDate >= awardCompletionDate)


dat = dat[~dat['reasonForModification'].isin(impossible_savings_statuses)]

dat['isNaturalCloseOut'] = dat.apply(lambda row: natural_closeout(
    pd.to_datetime(row['displayClosedDate'], format="%m/%d/%Y %H:%M:%S"),
    pd.to_datetime(row['awardCompletionDate'], format="%m/%d/%Y")), axis=1)

dat = dat[~(dat['reasonForModification'] == "Close Out") & (dat["isNaturalCloseOut"] == False)]

NUM_VALID_CANCELLATIONS = len(dat)
NUM_SAVINGS_VALID_CANCELLATIONS = sum(dat['maximumSavingsRealized'])
NUM_SAVINGS_VALID_CANCELLATIONS = format_as_money(NUM_SAVINGS_VALID_CANCELLATIONS)
NUM_SAVINGS_VALID_CANCELLATIONS

'$1,405,030,139.76'

In [70]:
rows = [
    {
        "Category": "DOGE's claimed savings in cancelled contracts",
        "Value of Savings": DOGE_CLAIMED_SAVINGS,
        "Number of Contracts": NUM_CONTRACTS_TOTAL,
        "Notes": "Sum of all the values on doge.gov 'Savings' tab on 2/22/2025"
    },
        {
        "Category": "After calculating actual savings",
        "Value of Savings": MAX_SAVINGS_ACTUALS,
        "Number of Contracts": NUM_CONTRACTS_TOTAL,
        "Notes": ""
    },
        {
        "Category": "After removing contracts that haven't changed since inaguration",
        "Value of Savings": MAX_SAVINGS_ACTUALS_AFTER_INAGURATION,
        "Number of Contracts": NUM_CONTRACTS_AFTER_INAGURATION,
        "Notes": ""
    },
        {
        "Category": "After removing contracts that were not actually cancelled",
        "Value of Savings": NUM_SAVINGS_VALID_CANCELLATIONS,
        "Number of Contracts": NUM_VALID_CANCELLATIONS,
        "Notes": ""
    }
]

summary = pd.DataFrame(rows)
summary.to_excel("savings_funnel.xlsx")